##### Run the script below in a new env because pyradiomics has forward compatibility issues and available for only python < 3.10
- conda create -n radiomics python=3.9
- conda install -c radiomics -c conda-forge pyradiomics --yes
- pip install nibabel SimpleITK pandas matplotlib jupyter lifelines
- 

In [1]:
import os
from collections import OrderedDict
import nibabel as nib
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
from scipy.ndimage import label
import pickle

def get_weighting_factor(peak_hu):
    """
    Return the weighting factor based on the lesion's peak HU value.
    """
    if 130 <= peak_hu < 200:
        return 1
    elif 200 <= peak_hu < 300:
        return 2
    elif 300 <= peak_hu < 400:
        return 3
    elif peak_hu >= 400:
        return 4
    else:
        return 0
    
def calculate_cac_score(ct_image, mask_image, min_pixels=3):
    """
    Calculate the CAC (Agatston) score for a given CT image and its corresponding coronary arteries mask.
    Parameters:
    ct_image (sitk image): The CT image in SimpleITK format.
    mask_image (sitk image): The coronary arteries mask in SimpleITK format.
    min_pixels (int): Minimum number of connected pixels to consider a lesion valid.
    
    Returns:
    float: The total Agatston score.
    """
    
    # Convert images to numpy arrays (shape: [slices, height, width])
    ct_array = sitk.GetArrayFromImage(ct_image).astype(np.float32)
    mask_array = sitk.GetArrayFromImage(mask_image).astype(np.uint8)
    
    # Get pixel spacing; Sitk returns spacing as (x, y, z)
    spacing = ct_image.GetSpacing()
    # Calculate pixel area in mm² in the axial plane
    pixel_area = spacing[0] * spacing[1]
    
    total_score = 0.0
    
    # Process each slice (the Agatston score is calculated slice by slice)
    for slice_idx in range(ct_array.shape[0]):
        ct_slice = ct_array[slice_idx]
        mask_slice = mask_array[slice_idx]
        
        # Perform connected component analysis on the mask slice
        labeled_slice, num_features = label(mask_slice, structure=np.ones((3, 3)))
        
        for i in range(1, num_features + 1):
            lesion_mask = (labeled_slice == i)
            if np.sum(lesion_mask) < min_pixels:
                continue  # Skip small lesions
            
            # Compute the lesion's area in mm² (number of pixels * pixel area)
            lesion_area = np.sum(lesion_mask) * pixel_area 
            
            # Compute the lesion's peak HU value
            lesion_peak_hu = np.max(ct_slice[lesion_mask])
            weighting = get_weighting_factor(lesion_peak_hu)
            
            # Lesion score is the product of area and weighting factor
            lesion_score = lesion_area * weighting
            total_score += lesion_score
            
    return total_score
    

In [2]:
# extract HRFs from data

test_patient_ids, test_radiomics = [], []
test_images_file_path = "D:\\Shruti\\IS_project\\nnunet_input_images"
test_masks_file_path = "D:\\Shruti\\IS_project\\preds_coronary_arteries"

for filename in sorted(os.listdir(test_images_file_path)):
    if filename.endswith('.nii.gz'):
        patient_id = filename.replace('_0000.nii.gz', '')
        test_patient_ids.append(patient_id)
        
        image_path = os.path.join(test_images_file_path, filename)
        mask_path = os.path.join(test_masks_file_path, filename)

        if not os.path.exists(mask_path):
            print(f"Mask not found for Patient ID: {patient_id}, skipping...")
            continue
        print(f"Processing Patient ID: {patient_id} with image: {image_path} and mask: {mask_path}")

        sitk_image = sitk.Cast(sitk.ReadImage(image_path), sitk.sitkFloat32)
        sitk_mask = sitk.Cast(sitk.ReadImage(mask_path), sitk.sitkUInt8)

        print(np.unique(sitk.GetArrayFromImage(sitk_mask)))

        try:
            score = calculate_cac_score(sitk_image, sitk_mask, min_pixels=1)
            print(f"Calculated CAC Score for Patient ID {patient_id}: {score}")
            features_series = dict(CA_score = score)
        except Exception as e:
            features_series = dict(CA_score = 0.0)
            
        features_series['patient_id'] = patient_id
        test_radiomics.append(features_series)

Processing Patient ID: 001_02_Baseline with image: D:\Shruti\IS_project\nnunet_input_images\001_02_Baseline_0000.nii.gz and mask: D:\Shruti\IS_project\preds_coronary_arteries\001_02_Baseline_0000.nii.gz
[0 1]
Calculated CAC Score for Patient ID 001_02_Baseline: 682.9475936889648
Processing Patient ID: 001_03_Baseline with image: D:\Shruti\IS_project\nnunet_input_images\001_03_Baseline_0000.nii.gz and mask: D:\Shruti\IS_project\preds_coronary_arteries\001_03_Baseline_0000.nii.gz
[0 1]
Calculated CAC Score for Patient ID 001_03_Baseline: 4042.2821044921875
Processing Patient ID: 001_04_Baseline with image: D:\Shruti\IS_project\nnunet_input_images\001_04_Baseline_0000.nii.gz and mask: D:\Shruti\IS_project\preds_coronary_arteries\001_04_Baseline_0000.nii.gz
[0 1]
Calculated CAC Score for Patient ID 001_04_Baseline: 2534.311222076416
Processing Patient ID: 001_05_Baseline with image: D:\Shruti\IS_project\nnunet_input_images\001_05_Baseline_0000.nii.gz and mask: D:\Shruti\IS_project\preds_co

In [3]:
test_radiomics_to_save = pd.DataFrame(test_radiomics)
test_radiomics_to_save

,CA_score,patient_id
0,682.947594,001_02_Baseline
1,4042.282104,001_03_Baseline
2,2534.311222,001_04_Baseline
3,1189.195496,001_05_Baseline
4,659.855530,001_07_Baseline
5,1881.079865,001_09_Baseline
6,1141.871033,001_11_Baseline
7,959.027679,001_12_Baseline
8,1677.141190,001_13_Baseline
9,3923.470184,001_14_Baseline


In [4]:
test_radiomics_to_save.to_csv('D:\\Shruti\\IS_project\\test_radiomics_CAC_scores.csv', index=False)